In [44]:
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, make_scorer, classification_report
import pandas as pd
import numpy as np
import random

In [45]:
def preprocessing_1(X):
    X_normalized = []
    for i in range(len(X)):
        X_normalized.append(np.array(X[i])/sum(X[i]))
    return X_normalized

In [46]:
def save_pickle(file_name, file_path):
    with open(file_path, 'wb') as fp:
        pickle.dump(file_name, fp)

In [47]:
def remove_nan(npi_indigo_spl):
    npi_indigo_spl_non_nan = {}
    for npi in npi_indigo_spl:
        if isinstance(npi_indigo_spl[npi], str):
            npi_indigo_spl_non_nan[npi] = npi_indigo_spl[npi]
    return npi_indigo_spl_non_nan

In [48]:
def create_dataset(npi_distributions, npi_indigo_spl):
    dataset = {'NPI':[], 'features':[], 'labels':[]}
    for npi in npi_distributions:
        if int(npi) in npi_indigo_spl:
            dataset['NPI'].append(npi)
            dataset['features'].append(npi_distributions[npi])
            dataset['labels'].append(npi_indigo_spl[int(npi)])
    return dataset

In [49]:
def get_less_than_threshold_categories(y, threshold):
    labels_count = {}
    for lab in y:
        if lab in labels_count:
            labels_count[lab] += 1
        else:
            labels_count[lab] = 1

    useless = []
    for lab in labels_count:
        if labels_count[lab] < threshold:
            useless.append(lab)
    return useless

In [50]:
def remove_useless(useless, y, X):
    for i in reversed(range(len(y))):
        if y[i] in useless:
            del y[i]
            del X[i]
    return X, y

In [51]:
#def encode_labels(y):
#    label_encoder = LabelEncoder()
#    return label_encoder.fit_transform(y), label_encoder

In [52]:
def sanity_check(y_train):
    check = set(y_train)

    print('Max value in input: ' +  str(max(check)))
    print('"length-1" of set of input: ' + str(len(check)-1))
    assert max(check) == (len(check)-1)

In [53]:
def remove_from_y(X_train_, y_train_, remove_list):
    
    X_train_rm = []
    y_train_rm = []

    for i in range(len(X_train_)):
        if y_train_[i] not in remove_list:
            X_train_rm.append(X_train_[i])
            y_train_rm.append(y_train_[i])
    return X_train_rm, y_train_rm

In [54]:
def check_dataset_distribution(y_train, k):
    dataset_distribution = {}
    total = 0
    for i in range(len(y_train)):
        if y_train[i] not in dataset_distribution:
            dataset_distribution[y_train[i]] = 1
        else:
            dataset_distribution[y_train[i]] += 1
        total += 1
    
    print(dataset_distribution)
    remove_list = []
    for key in dataset_distribution:
        if dataset_distribution[key] < k:
            remove_list.append(key)
    
    print(list(dataset_distribution.values()))
    print(max(list(dataset_distribution.values())))
    print(min(list(dataset_distribution.values())))
    print(np.std(list(dataset_distribution.values())))
    return remove_list

In [55]:
def encode_labels_2way(y):
    encoded_y = []
    for l in y:
        if '-Surgery' in l or '-Minor' in l:
            encoded_y.append(0)
        else:
            encoded_y.append(1)
    return encoded_y

In [56]:
def encode_labels_2way_custom(y, label):
    encoded_y = []
    for l in y:
        if label in l:
            encoded_y.append(0)
        else:
            encoded_y.append(1)
    return encoded_y

In [188]:
def overpopulate(X, y, op_multiplier, op_ratio_retained):
    X_oped = []
    y_oped = []

    for i in range(len(y)):
        if y[i] == 0:
            for _ in range(op_multiplier):
                X_oped.append(X[i])
                y_oped.append(y[i])
        elif random.uniform(0, 1) < op_ratio_retained:
            X_oped.append(X[i])
            y_oped.append(y[i])
    
    return X_oped, y_oped

In [58]:
def get_top_k_cpts_with_multipliers(normalized_distributions, k):
    
    all_cpt_pos = {}
    for i in range(len(normalized_distributions)):
        cur_top_agg = normalized_distributions[i].argsort()[-k:][::-1]
        for pos in cur_top_agg:
            if pos not in all_cpt_pos:
                all_cpt_pos[pos] = 0
            all_cpt_pos[pos] += 1
                
    for cpt_pos in all_cpt_pos:
        all_cpt_pos[cpt_pos] = np.log(len(normalized_distributions)/all_cpt_pos[cpt_pos])
    
    return all_cpt_pos

In [175]:
def get_top_k_cpts(normalized_distributions, k, tfidf = True):
    
    all_cpt_pos = {}
    print("Printing CPT codes with speciality")
    for i in range(len(normalized_distributions)):
        cur_top_agg = normalized_distributions[i].argsort()[-k:][::-1]
        for pos in cur_top_agg:
            if pos not in all_cpt_pos:
                all_cpt_pos[pos] = 0
            all_cpt_pos[pos] += 1
    
    single = [code for code in all_cpt_pos if all_cpt_pos[code] == 1]; double = [code for code in all_cpt_pos if all_cpt_pos[code] == 2]
    print(single)
    print(double)
    print(len(double) + len(single))

    if tfidf:
        for cpt_pos in all_cpt_pos:
            all_cpt_pos[cpt_pos] = np.log(len(normalized_distributions)/all_cpt_pos[cpt_pos])

        
    return all_cpt_pos

In [60]:
def get_idf_vector(all_cpt_pos, num_featurs):
    idf_vector = []
    for i in range(num_featurs):
        if i in all_cpt_pos:
            idf_vector.append(all_cpt_pos[i])
        else:
            idf_vector.append(0)

    return idf_vector

In [61]:
def normalize(X):
    for i in range(len(X)):
        X[i] = X[i]/sum(X[i])
    return X

In [62]:
def get_distributions(X, y):
    distributions = [np.array(len(X[0])*[0.0]) for _ in range(len(set(y)))]
    for i in range(len(X)):
        # if i%10000 == 0:
        #     print("Done: " + str(i))
        distributions[y[i]] += X[i]
    
    distributions[1] += distributions[0]
    
    distributions = normalize(distributions)

    return distributions

In [63]:
def get_tfidf(tf_matrix, idf_vector):
    tfidfs = []
    for tf in tf_matrix:
        tfidfs.append(np.multiply(tf, idf_vector[0]))
    return tfidfs

In [64]:
def reduce_dimention(all_cpt_idfs, X_normalized):
    cpt_pos = list(all_cpt_idfs.keys())
    #print(cpt_pos)

    X_reduced = []

    for distribution in X_normalized:
        temp = []
        for pos in cpt_pos:
            if all_cpt_idfs[pos] >= 0.001:
                temp.append(distribution[pos])
        X_reduced.append(temp)
    
    return X_reduced

In [22]:
npi_features = pd.read_pickle('./chuncked_npi_ncpcs_2019_0_.pkl')

In [23]:
npi_indigo_spl = pd.read_pickle('./npi_indigo_spl.pkl')
npi_indigo_spl_non_nan = remove_nan(npi_indigo_spl)

In [24]:
dataset = create_dataset(npi_features, npi_indigo_spl_non_nan)

In [25]:
X = dataset['features']
y = dataset['labels']

In [26]:
print(len(X))
print(len(y))

153954
153954


Note: Maybe try a binary model of seperating surgical and non-surgical doctors and then create further indigo speciality distinctions for each of these 2 classes.

Note: Create a binary model each indigo speciality (True / False). Kind of a multi-label classification model.

In [65]:
def get_results(random_search, results_test, results_train, X_val, y_val, X_train, y_train, spl):
    predicted_probabilities = random_search.best_estimator_.predict_proba(X_val)
    predictions_test = np.argmax(predicted_probabilities, axis=1)

    predicted_probabilities = random_search.best_estimator_.predict_proba(X_train)
    predictions_train = np.argmax(predicted_probabilities, axis=1)

    test_out = classification_report(y_val, predictions_test, target_names=[spl, 'Others'], output_dict=True)
    train_out = classification_report(y_train, predictions_train, target_names=[spl, 'Others'], output_dict=True)

    results_train.append(["CPT upper level features", "Train", spl, train_out['Others']['precision'], train_out['Others']['recall'], train_out[spl]['precision'], train_out[spl]['recall'], train_out['macro avg']['f1-score']])
    results_test.append(["CPT upper level features", "Test", spl, test_out['Others']['precision'], test_out['Others']['recall'], test_out[spl]['precision'], test_out[spl]['recall'], test_out['macro avg']['f1-score']])

    return results_train, results_test

In [199]:
def spit_train_test_split(X, y, speciality, param_dist, results_test, results_train, k=25, tfidf=True, majority_class_data_split=0.1, minority_class_op_coef=1, test_size=0.3, random_state=897):
    encoded_y = encode_labels_2way_custom(y, speciality)
    encoded_y_np = np.array(encoded_y)

    if len(encoded_y_np[encoded_y_np==0]) > 0:

        X_oped, y_oped = overpopulate(X, encoded_y, minority_class_op_coef, majority_class_data_split)
        X_oped_normalized = preprocessing_1(X_oped)

        normalized_class_distributions = get_distributions(X_oped_normalized, y_oped)
        all_cpt_pos_idf = get_top_k_cpts(normalized_class_distributions, k, tfidf)
        idf_vector = get_idf_vector(all_cpt_pos_idf, len(X_oped_normalized[0]))

        X_oped_normalized_reduced = reduce_dimention(all_cpt_pos_idf, X_oped_normalized)
        # print(np.array(X_oped_normalized_reduced).shape)
        idf_vector_reduced = reduce_dimention(all_cpt_pos_idf, [idf_vector])
        # print(idf_vector_reduced)

        X_oped_normalized_reduced_tfidfs = get_tfidf(X_oped_normalized_reduced, idf_vector_reduced)
        X_oped_normalized_reduced_tfidfs = np.array(X_oped_normalized_reduced_tfidfs)
        
        X_train, X_val, y_train, y_val = train_test_split(X_oped_normalized_reduced_tfidfs, y_oped, test_size=test_size, random_state=random_state, stratify=y_oped, shuffle=True)
        # X_train, y_train = overpopulate(X_train, y_train, minority_class_op_coef, 1)
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        print(X_train.shape)
        y_train = np.array(y_train)
        print(len(y_train[y_train==0]), len(y_train[y_train==1]))
        y_val = np.array(y_val)
        print(len(y_val[y_val==0]), len(y_val[y_val==1]))

        model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(set(y_train)), tree_method='gpu_hist', gpu_id=0)
        random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=4, scoring='accuracy', n_jobs=-1, cv=2, verbose=3, random_state=53)
        random_search.fit(X_train, y_train)
        print("Best Estimator: ", random_search.best_params_)
        results_train, results_test = get_results(random_search, results_test, results_train, X_val, y_val, X_train, y_train, speciality)
        model = None
        random_search = None
        print('\033[1m' + '\033[92m' + "Done: " + speciality + '\033[0m' + '\033[0m')
    else:
        print('\033[1m' + '\033[91m' + speciality + " not in CPT upper level features" + '\033[0m' + '\033[0m')
    
    return results_train, results_test


In [1]:
# specialities_lis = pd.read_pickle("./specialities_lis.pkl")
# specialities_lis = ["Cardiovascular Disease-Minor Surgery"]
specialities_lis = [
    "Clinical Nurse Specialist",
    "Certified Nurse Midwife (CNM)",
    "Anesthesiology Assistant (AA)",
    "Cardiovascular Disease-Surgery",
    "Vascular-Surgery",
    "Dermatology-Minor Surgery",
    "Neurology-Surgery",
    "Gynecology-Minor Surgery",
    "Gynecology-No Surgery",
    "General Preventive Med-No Surgery",
    "Orthopedic Incl Back-Surgery",
    "Ophthalmology-Minor Surgery",
    "Ophthalmology-Surgery",
    "Otorhinolaryngology-No Surgery",
    "Otorhinolaryngology-Surgery",
    "Radiology Diagnostic-Minor Surgery",
    "Pediatrics-Minor Surgery",
    "Aerospace Medicine",
    "Bariatric-Surgery",
    "Colon And Rectal-Surgery",
    "Emergency Medical Technician (EMT)",
    "Hand-Surgery",
    "Forensic Medicine",
    "Geriatrics-No Surgery",
    "Hand-Surgery",
    "Nuclear Medicine",
    "O.R. Technician",
    "Occupational Medicine",
    "Pharmacist",
    "Phlebology",
    "Physicians NOC-No Surgery",
    "Physiotherapist",
    "Plastic Otorhinolaryngology-Surgery",
    "Respiratory Therapist",
    "Thoracic-Surgery",
    "Traumatic-Surgery",
    "Urgent Care-No Surgery"
]

In [179]:
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 7, 8, 9],
    'colsample_bytree': [0.6, 0.65, 0.7, 0.75],
}

results_test = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
results_train = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
for spl in specialities_lis:
    results_train, results_test = spit_train_test_split(X, y, spl, param_dist, results_test, results_train)
    print("Done: ", spl)
    break

df = pd.DataFrame(results_test + results_train)
df
#df.to_csv('./binary_spl_prediction_tfidf_top25_output_v1.csv', index=False)

Printing CPT codes with speciality
[7461, 7423, 7438, 6284, 11777, 1482, 6283, 6089, 7429, 6081, 15706, 1483, 239, 238, 6075, 14456, 236, 14896, 14460, 12571, 189, 191, 14457, 190]
[7393, 7392, 14462, 6073, 6080, 2450, 14461, 407, 2451, 7394, 6074, 12084, 13927]
37
(11057, 24)
192 10865
82 4658
Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Best Estimator:  {'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.75}
Done: Geriatrics-No Surgery
Done:  Geriatrics-No Surgery


,0,1,2,3,4,5,6,7
0,Data Type,Train / Test,Speciality,Precision 0,Recall 0,Precision 1,Recall 1,Macro Avg F1
1,CPT upper level features,Test,Geriatrics-No Surgery,0.983305,0.998927,0.375,0.036585,0.52886
2,Data Type,Train / Test,Speciality,Precision 0,Recall 0,Precision 1,Recall 1,Macro Avg F1
3,CPT upper level features,Train,Geriatrics-No Surgery,0.995784,1.0,1.0,0.760417,0.930896


In [193]:
specialities_lis = ["Occupational Medicine"]

In [204]:
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 7, 8, 9],
    'colsample_bytree': [0.6, 0.65, 0.7, 0.75],
}

results_test = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
results_train = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
for spl in specialities_lis:
    results_train, results_test = spit_train_test_split(X, y, spl, param_dist, results_test, results_train, k=25, tfidf=True, majority_class_data_split=0.05, minority_class_op_coef=4)
    print("Done: ", spl)

df = pd.DataFrame(results_test + results_train)
df
#df.to_csv('./binary_spl_prediction_tfidf_top25_output_v1.csv', index=False)

Printing CPT codes with speciality
[14464, 1493, 14500, 15762, 12574, 9429, 14507, 10228, 3238, 14236, 14238, 14458, 14502, 14463, 14580, 14484, 239, 238, 2450, 6080, 2451, 12084, 236, 7393, 14896, 191, 189, 13927, 12571, 7394, 190, 7392]
[14461, 14462, 14456, 6073, 14457, 407, 6075, 14460, 6074]
41
(5547, 32)
188 5359
20 2297
Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Best Estimator:  {'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.75}
Done: Occupational Medicine
Done:  Occupational Medicine


,0,1,2,3,4,5,6,7
0,Data Type,Train / Test,Speciality,Precision 0,Recall 0,Precision 1,Recall 1,Macro Avg F1
1,CPT upper level features,Test,Occupational Medicine,0.992211,0.998259,0.333333,0.1,0.574536
2,Data Type,Train / Test,Speciality,Precision 0,Recall 0,Precision 1,Recall 1,Macro Avg F1
3,CPT upper level features,Train,Occupational Medicine,0.994796,0.998694,0.958084,0.851064,0.949075


In [ ]:
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 7, 8, 9],
    'colsample_bytree': [0.6, 0.65, 0.7, 0.75],
}

k_range = np.arange(50, 700, 100)
runs_per_speciality_per_k = 3

results_test = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
results_train = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
for spl in specialities_lis:
    for k in range(k_range):
        results_train, results_test = spit_train_test_split(X, y, spl, param_dist, results_test, results_train, k=25, tfidf=True, majority_class_data_split=0.05, minority_class_op_coef=4)
    print("Done: ", spl)

df = pd.DataFrame(results_test + results_train)
df
#df.to_csv('./binary_spl_prediction_tfidf_top25_output_v1.csv', index=False)

In [184]:
cpt_codes_2019 = pd.read_pickle("./grouped_hcpcs_codes2019.pkl")
singles = [14464, 1493, 14500, 14457, 15762, 12574, 9429, 14507, 14460, 239, 238, 2450, 6080, 2451, 12084, 6074, 7393, 236]
singles_code = []
doubles = [14461, 14462, 14456, 6073, 407, 6075]
doubles_code = []
for pos in doubles:
    doubles_code.append(cpt_codes_2019[pos])
for pos in singles:
    singles_code.append(cpt_codes_2019[pos])

In [185]:
doubles_code

['99213', '99214', '99203', '97110', '36415', '97140']

In [186]:
singles_code

['99202',
 'S9083',
 '70450',
 '99204',
 '93798',
 '90471',
 '94010',
 '73030',
 '99212',
 '90837',
 '90834',
 '99232',
 '97530',
 '99233',
 '85025',
 '97112',
 'G0299',
 '90791']

In [80]:
cpt_codes_2019 = pd.read_pickle("./grouped_hcpcs_codes2019.pkl")
singles = [15705, 15714, 15713, 15712, 15703, 15762, 12086, 15755, 15733, 3837, 15715, 10815, 8934, 15753, 2468, 2466, 15758, 2463, 11766, 15711, 15708, 10819, 15735, 15731, 6073, 239, 238, 6075, 6080, 7393, 6074, 236, 13927, 189, 14896, 12571, 191, 190, 237, 7392, 14502, 176, 12574, 244, 7150, 10465, 2437, 10455]
singles_code = []
doubles = [15706, 14462, 15704, 14461, 2450, 2451, 407, 14457, 7394, 14463, 13929, 12084, 14456, 3446, 2449, 14460]
doubles_code = []
for pos in doubles:
    doubles_code.append(cpt_codes_2019[pos])
for pos in singles:
    singles_code.append(cpt_codes_2019[pos])

In [69]:
doubles_code

['93010',
 '99214',
 '93306',
 '99213',
 '99232',
 '99233',
 '36415',
 '99204',
 'G0463',
 '99215',
 '80048',
 '85025',
 '99203',
 '99223',
 '99231',
 '99212']

In [70]:
singles_code

['93000',
 '93325',
 '93320',
 '93303',
 '93005',
 '93798',
 '85610',
 '93321',
 '93296',
 '99152',
 '93458',
 '99244',
 '78452',
 '93304',
 'J1644',
 'J3010',
 '93018',
 'J2250',
 'Q9967',
 '93017',
 '93280',
 '99243',
 '93299',
 '93294',
 '97110',
 '90837',
 '90834',
 '97140',
 '97530',
 'G0299',
 '97112',
 '90791',
 '80053',
 '99284',
 '92507',
 '98941',
 '99283',
 '99285',
 '90832',
 'G0151',
 '71046',
 '77067',
 '90471',
 '90853',
 '88305',
 '92014',
 '99291',
 '92015']

In [42]:
doubles_code

['93010', '99214', '99213', '99232', '99233', '36415', '99204', 'G0463']

In [43]:
singles_code

['93306',
 '93000',
 '93325',
 '93320',
 '93303',
 '93005',
 '93798',
 '85610',
 '99215',
 '93321',
 '93296',
 '80048',
 '99152',
 '93458',
 '99244',
 '78452',
 '93304',
 '97110',
 '90837',
 '90834',
 '97140',
 '97530',
 '99203',
 '85025',
 '97112',
 'G0299',
 '99212',
 '90791',
 '80053',
 '92507',
 '99283',
 '98941',
 '99284',
 '99285']